### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import urllib.parse

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
api_key = tmdb_api_key

query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}")

print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=1b5zds6gWGokXaLgBHuj9hd6jSCI66lE&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(3):
    # create query with a page number
    page_url = f"{query_url}&page={page}"
    # API results show 10 articles at a time
    print(f"Retrieving page {page} of reviews...")
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Page {page} retrieved") 

    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page}")
        break

Retrieving page 0 of reviews...


Page 0 retrieved
Retrieving page 1 of reviews...
Page 1 retrieved
Retrieving page 2 of reviews...
Page 2 retrieved


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews_list)

In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df['title'] = df['headline.main'].str.extract("\u2018(.+?)\u2019 Review")

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['keywords'].apply(extract_keywords)

In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df['title'].to_list()

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [15]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter > 0 and request_counter % 50 == 0:
        print("Sleeping for 5 seconds...")
        time.sleep(5)
    # Add 1 to the request counter
    request_counter += 1
    # Perform a "GET" request for The Movie Database
    response = requests.get(f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}")

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        movie_id = response.json()["results"][0]["id"]
        movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}")
        movie_data = movie_response.json()
        
        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_data["genres"]]


        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_data["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_data["production_countries"]]
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data_dict = {
            "title": movie_data["title"],
            "release_date": movie_data["release_date"],
            "runtime": movie_data["runtime"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        tmdb_movies_list.append(movie_data_dict)
        # Print out the title that was found
        print(f"Movie found: {title}")

    except IndexError:
        # Print out the title that was not found
        print(f"Movie not found: {title}")
        break
        

Movie found: The Attachment Diaries
Movie found: nan
Movie found: You Can Live Forever
Movie found: A Tourist’s Guide to Love
Movie found: Other People’s Children
Movie found: One True Loves
Movie found: The Lost Weekend: A Love Story
Movie found: A Thousand and One
Movie found: Your Place or Mine
Movie found: Love in the Time of Fentanyl
Movie found: Pamela, a Love Story
Movie found: In From the Side
Movie found: After Love
Movie found: Alcarràs
Movie found: Nelly & Nadine
Movie found: Lady Chatterley’s Lover
Movie found: The Sound of Christmas
Movie found: The Inspection
Movie found: Bones and All
Movie found: My Policeman
Movie found: About Fate
Movie found: Waiting for Bojangles
Movie found: I Love My Dad
Movie found: A Love Song
Movie found: Alone Together
Movie found: Art of Love
Movie found: The Wheel
Movie found: Thor: Love and Thunder
Movie found: Both Sides of the Blade
Movie found: Fire of Love


In [16]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "release_date": "2021-10-07",
        "runtime": 102,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "Bread",
        "release_date": "2008-09-08",
        "runtime": 30,
        "genres": [
            "Drama",
            "Crime"
        ],
        "spoken_languages": [
            "Dutch",
            "Kurdish"
        ],
        "production_countries": [
            "Belgium",
            "Iraq"
        ]
    },
    {
        "title": "You Can Live Forever",
        "release_date": "2023-03-24",
        "runtime": 96,
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "production_

In [17]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df


,title,release_date,runtime,genres,spoken_languages,production_countries
0,The Attachment Diaries,2021-10-07,102,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,Bread,2008-09-08,30,"[Drama, Crime]","[Dutch, Kurdish]","[Belgium, Iraq]"
2,You Can Live Forever,2023-03-24,96,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,2023-04-21,96,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People's Children,2022-09-21,104,"[Drama, Comedy]","[English, French]",[France]
5,One True Loves,2023-04-07,100,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
6,The Lost Weekend: A Love Story,2023-04-13,95,[Documentary],[English],[United States of America]
7,A Thousand and One,2023-03-31,116,"[Drama, Crime]","[English, Portuguese, Spanish]",[United States of America]
8,Your Place or Mine,2023-02-10,109,"[Romance, Comedy]",[English],[United States of America]
9,Love in the Time of Fentanyl,2023-02-03,85,[Documentary],[English],"[Canada, United States of America]"


### Merge and Clean the Data for Export

In [18]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df, tmdb_df, on="title", how="left")

In [19]:
column_names = merged_df.columns
print(column_names)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title', 'release_date', 'runtime', 'genres',
       'spoken_languages', 'production_countries'],
      dtype='object')


In [20]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries", "keywords"]

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    print(column)

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)




    # Loop through characters to remove
    for character in characters_to_remove:

        # Remove the character from the column
        merged_df[column] = merged_df[column].str.replace(character, "")




# Display the fixed DataFrame
merged_df.head()

genres
spoken_languages
production_countries
keywords


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.person,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,2021-10-07,102.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,2023-03-24,96.0,"Drama, Romance","English, French","Canada, United States of America"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,NaN,NaN,nan,nan,nan


In [21]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries', 'keywords']

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        # Remove the character from the column
        merged_df[column] = merged_df[column].str.replace(character, "")

# Display the fixed DataFrame
merged_df.head()



# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]


# Loop through the list of columns to fix
for column in columns_to_fix:
    print(column)

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)


    # Loop through characters to remove
    for character in characters_to_remove:

        # Remove the character from the column
        merged_df[column] = merged_df[column].str.replace(character, "")


# Display the fixed DataFrame
merged_df.head()


genres
spoken_languages
production_countries
keywords


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.person,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,2021-10-07,102.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,2023-03-24,96.0,"Drama, Romance","English, French","Canada, United States of America"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,NaN,NaN,nan,nan,nan


In [22]:
# Drop "byline.person" column
if "byline.person" in merged_df.columns:
    merged_df = merged_df.drop(columns=["byline.person"])

merged_df.head(20)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,2021-10-07,102.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,None,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,2023-03-24,96.0,"Drama, Romance","English, French","Canada, United States of America"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,None,Other People’s Children,NaN,NaN,nan,nan,nan
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,None,None,By Brandon Yu,None,One True Loves,2023-04-07,100.0,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,None,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,2023-04-13,95.0,Documentary,English,United States of America
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,None,None,By Manohla Dargis,None,A Thousand and One,2023-03-31,116.0,"Drama, Crime","English, Portuguese, Spanish",United States of America
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,None,None,By Amy Nicholson,None,Your Place or Mine,2023-02-10,109.0,"Romance, Comedy",English,United States of America
9,https://www.nytimes.com/2023/02/02/movies/love...,"To com

In [23]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,2021-10-07,102.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,None,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,2023-03-24,96.0,"Drama, Romance","English, French","Canada, United States of America"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,None,Other People’s Children,NaN,NaN,nan,nan,nan


In [25]:
# Export data to CSV without the index
merged_df.to_csv("output/nyt_movie_reviews.csv", index=False)
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.organization,title,release_date,runtime,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,2021-10-07,102.0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,None,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,2023-03-24,96.0,"Drama, Romance","English, French","Canada, United States of America"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,None,Other People’s Children,NaN,NaN,nan,nan,nan
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,None,None,By Brandon Yu,None,One True Loves,2023-04-07,100.0,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,None,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,2023-04-13,95.0,Documentary,English,United States of America
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,None,None,By Manohla Dargis,None,A Thousand and One,2023-03-31,116.0,"Drama, Crime","English, Portuguese, Spanish",United States of America
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,None,None,By Amy Nicholson,None,Your Place or Mine,2023-02-10,109.0,"Romance, Comedy",English,United States of America
9,https://www.nytimes.com/2023/02/02/movies/love...,"To com